# Overview

In this notebook, I conduct the design and development of artificial intelligence models. The main goal of it is to try to predict the physical exercise adherence of the users based on how much time they have been exercising. 

**Author**: Jon Maestre Escobar

**Email**: jonmaestre@opendeusto.es.

In [1]:
import pandas as pd
import numpy as np
import warnings
import pandas as pd
from utilities import Data_cleaning

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.express.colors import sample_colorscale

import math
import copy
import re
%matplotlib inline

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
df_filtered_merged = pd.read_hdf('../data/filtered_merged_dataset_v1.h5', key='df')
df_filtered_merged.fillna(0, inplace=True)
df_filtered_merged.shape

(46020, 8705)

## **Users Filtered by Different Training Periods**

In this section, I have undertaken a detailed process to filter and analyze users based on the duration of their training periods recorded in our dataset. My objective was to categorize users into different groups based on their training consistency and ensure comprehensive data analysis by focusing on the range of days from the first to the last recorded training session.

**Steps Taken**

- **Initial Filtering**:
    - **30 Days or Less**: I first identified users whose training period spans 30 days or less from their first recorded training session to their last.
    - **31 to 90 Days**: Next, I filtered users whose training period spans more than 30 days but not exceeding 90 days.
    - **91 to 180 Days**: Similarly, I filtered users whose training period spans more than 90 days but up to 180 days.
    - **181 to 365 Days**: Finally, I identified users whose training period spans more than 180 days but not exceeding 365 days.

- **Data Completion**:
    - For each of these groups, I ensured that the training records were complete. Specifically, I filled in any missing days with zeros from the user's first recorded training day to ensure there are no gaps in the data.
    - This involved generating a complete date range for each user based on their first training session and merging it with the existing records, filling in the missing entries.

- **Detailed Analysis**:
    - The filtered and completed datasets were then analyzed to understand user behavior and training patterns better. This step helps in identifying trends and making data-driven recommendations for improving user engagement and training adherence.

By implementing these steps, I aim to provide a robust analysis framework that accurately reflects users' training habits, allowing us to draw meaningful insights and support the development of personalized fitness interventions. This comprehensive approach ensures that our analysis accounts for all training days, even those that were not initially recorded, providing a more accurate picture of user activity.

In [3]:
# Convert 'date' to datetime
df_filtered_merged['date'] = pd.to_datetime(df_filtered_merged['date'])

# Calculate the first and last training day for each user
user_training_period = df_filtered_merged.groupby('user_programs_user_id')['date'].agg(['min', 'max']).reset_index()
user_training_period.columns = ['user_programs_user_id', 'first_training_date', 'last_training_date']

# Calculate the difference in days between the first and last training session
user_training_period['training_period_days'] = (user_training_period['last_training_date'] - user_training_period['first_training_date']).dt.days

# Filter users whose training period is 30 days or less
users_30_days_or_less = user_training_period[user_training_period['training_period_days'] <= 30]

# Create a DataFrame with the filtered users
df_filtered_30days = df_filtered_merged[df_filtered_merged['user_programs_user_id'].isin(users_30_days_or_less['user_programs_user_id'])]
df_filtered_30days.shape

(4596, 8705)

In [4]:
# Convert 'date' to datetime
df_filtered_merged['date'] = pd.to_datetime(df_filtered_merged['date'])

# Calculate the first and last training day for each user
user_training_period = df_filtered_merged.groupby('user_programs_user_id')['date'].agg(['min', 'max']).reset_index()
user_training_period.columns = ['user_programs_user_id', 'first_training_date', 'last_training_date']

# Calculate the difference in days between the first and last training session
user_training_period['training_period_days'] = (user_training_period['last_training_date'] - user_training_period['first_training_date']).dt.days

# Filter users whose training period is between 30 and 90 days
users_30_to_90_days = user_training_period[(user_training_period['training_period_days'] > 30) & (user_training_period['training_period_days'] <= 90)]

# Create a DataFrame with the filtered users
df_filtered_30_to_90_days = df_filtered_merged[df_filtered_merged['user_programs_user_id'].isin(users_30_to_90_days['user_programs_user_id'])]
df_filtered_30_to_90_days.shape

(7006, 8705)

In [5]:
# Convert 'date' to datetime
df_filtered_merged['date'] = pd.to_datetime(df_filtered_merged['date'])

# Calculate the first and last training day for each user
user_training_period = df_filtered_merged.groupby('user_programs_user_id')['date'].agg(['min', 'max']).reset_index()
user_training_period.columns = ['user_programs_user_id', 'first_training_date', 'last_training_date']

# Calculate the difference in days between the first and last training session
user_training_period['training_period_days'] = (user_training_period['last_training_date'] - user_training_period['first_training_date']).dt.days

# Filter users whose training period is between 90 and 180 days
users_90_to_180_days = user_training_period[(user_training_period['training_period_days'] > 90) & (user_training_period['training_period_days'] <= 180)]

# Create a DataFrame with the filtered users
df_filtered_90_to_180_days = df_filtered_merged[df_filtered_merged['user_programs_user_id'].isin(users_90_to_180_days['user_programs_user_id'])]
df_filtered_90_to_180_days.shape

(13243, 8705)

In [6]:
# Convert 'date' to datetime
df_filtered_merged['date'] = pd.to_datetime(df_filtered_merged['date'])

# Calculate the first and last training day for each user
user_training_period = df_filtered_merged.groupby('user_programs_user_id')['date'].agg(['min', 'max']).reset_index()
user_training_period.columns = ['user_programs_user_id', 'first_training_date', 'last_training_date']

# Calculate the difference in days between the first and last training session
user_training_period['training_period_days'] = (user_training_period['last_training_date'] - user_training_period['first_training_date']).dt.days

# Filter users whose training period is between 180 and 365 days
users_180_to_365_days = user_training_period[(user_training_period['training_period_days'] > 180) & (user_training_period['training_period_days'] <= 365)]

# Create a DataFrame with the filtered users
df_filtered_180_to_365_days = df_filtered_merged[df_filtered_merged['user_programs_user_id'].isin(users_180_to_365_days['user_programs_user_id'])]
df_filtered_180_to_365_days.shape

(21175, 8705)

In [7]:
print(f'The number of user who have trained 30 days or less is:', df_filtered_30days.user_programs_user_id.nunique()) 
print(f'The number of user who have trained between 31 and 90 days is:', df_filtered_30_to_90_days.user_programs_user_id.nunique())
print(f'The number of user who have trained between 91 and 180 days is', df_filtered_90_to_180_days.user_programs_user_id.nunique())
print(f'The number of user who have trained between 181 and 365 days is', df_filtered_180_to_365_days.user_programs_user_id.nunique())

The number of user who have trained 30 days or less is: 1658
The number of user who have trained between 31 and 90 days is: 549
The number of user who have trained between 91 and 180 days is 572
The number of user who have trained between 181 and 365 days is 409


### **Complete With Zeros Non-Training Days**

In this part of the analysis, I focused on ensuring that each user's training record is complete by filling in non-training days with zeros. This step is crucial for accurate time series analysis and to avoid any misinterpretation of the user's training consistency. Here are the detailed steps I followed:

- **Identify First and Last Training Day**:
    - For each user in the filtered datasets, I identified the first and last day they recorded a training session. This serves as the basis for generating a complete date range for each user.

- **Generate Complete Date Ranges**:
    - Using the first training day as the starting point, I generated a complete date range for each user up to the specified number of days (30 days for users with up to 30 days of training, 90 days for users with training periods of 31-90 days, 180 days for users with training periods of 91-180 days, and 365 days for users with training periods of 181-365 days).

- **Merge with Original Data and Fill Missing Days**:
    - I merged these generated date ranges with the original user training data to identify days that were not recorded.
    - For the missing days, I filled in the relevant columns with zeros, ensuring that there are no gaps in the training data.

- **Combine Completed Data**:
    - The completed data for each user was then combined into a single DataFrame. This new DataFrame includes all the original training data as well as the newly added rows for the non-training days filled with zeros.

This process ensures that each user has a continuous record of training activity, allowing for more accurate analysis and modeling. By filling in the non-training days with zeros, I can better understand user behavior and training patterns, which is essential for developing effective AI models and making informed recommendations.

In [8]:
# Ensure that 'date' is in datetime format
df_filtered_30days['date'] = pd.to_datetime(df_filtered_30days['date'])

# Get the first training day for each user
first_training_day = df_filtered_30days.groupby('user_programs_user_id')['date'].min().reset_index()
first_training_day.columns = ['user_programs_user_id', 'first_training_date']

# Create an empty DataFrame to store the completed data
completed_data_30days = pd.DataFrame()

# Iterate over each user and complete the missing days
for user_id, first_day in zip(first_training_day['user_programs_user_id'], first_training_day['first_training_date']):
    # Generate a date range from the first day up to 30 days later
    date_range = pd.date_range(start=first_day, periods=30)
    
    # Create a DataFrame with the date range and user_id
    user_dates = pd.DataFrame({'user_programs_user_id': user_id, 'date': date_range})
    
    # Merge with the original DataFrame to identify the days with and without training
    user_data = pd.merge(user_dates, df_filtered_30days[df_filtered_30days['user_programs_user_id'] == user_id], on=['user_programs_user_id', 'date'], how='left')
    
    # Fill NaN values (days without training) with zeros in the relevant columns
    user_data.fillna(0, inplace=True)
    
    # Add the completed data to the final DataFrame
    completed_data_30days = pd.concat([completed_data_30days, user_data])

# Reset the index of the final DataFrame
completed_data_30days.reset_index(drop=True, inplace=True)
print(completed_data_30days)

       user_programs_user_id       date session_executions_updated_at  \
0                        172 2021-11-13    2021-11-13 10:11:42.357218   
1                        172 2021-11-14    2021-11-14 12:05:10.670652   
2                        172 2021-11-15    2021-11-15 12:53:48.363087   
3                        172 2021-11-16                             0   
4                        172 2021-11-17                             0   
...                      ...        ...                           ...   
49735                  18174 2022-06-20                             0   
49736                  18174 2022-06-21                             0   
49737                  18174 2022-06-22                             0   
49738                  18174 2022-06-23                             0   
49739                  18174 2022-06-24                             0   

       1 leg bridge (left)_reps_1  1 leg bridge (left)_reps_10  \
0                             0.0                        

In [9]:
completed_data_30days.to_hdf('../data/completed_data_30days_v1.h5', key='df')

In [10]:
completed_data_30days = pd.read_hdf('../data/completed_data_30days_v1.h5', key='df')
print(f'30 days dataset shape:', completed_data_30days.shape)

30 days dataset shape: (49740, 8705)


In [29]:
# Ensure that 'date' is in datetime format
df_filtered_30_to_90_days['date'] = pd.to_datetime(df_filtered_30_to_90_days['date'])

# Get the first training day for each user
first_training_day_30_to_90 = df_filtered_30_to_90_days.groupby('user_programs_user_id')['date'].min().reset_index()
first_training_day_30_to_90.columns = ['user_programs_user_id', 'first_training_date']

# Create an empty DataFrame to store the completed data
completed_data_30_to_90_days = pd.DataFrame()

# Iterate over each user and complete the missing days
for user_id, first_day in zip(first_training_day_30_to_90['user_programs_user_id'], first_training_day_30_to_90['first_training_date']):
    # Generate a date range from the first day up to 90 days later
    date_range = pd.date_range(start=first_day, periods=90)
    
    # Create a DataFrame with the date range and user_id
    user_dates = pd.DataFrame({'user_programs_user_id': user_id, 'date': date_range})
    
    # Merge with the original DataFrame to identify the days with and without training
    user_data = pd.merge(user_dates, df_filtered_30_to_90_days[df_filtered_30_to_90_days['user_programs_user_id'] == user_id], on=['user_programs_user_id', 'date'], how='left')
    
    # Fill NaN values (days without training) with zeros in the relevant columns
    user_data.fillna(0, inplace=True)
    
    # Add the completed data to the final DataFrame
    completed_data_30_to_90_days = pd.concat([completed_data_30_to_90_days, user_data])

# Reset the index of the final DataFrame
completed_data_30_to_90_days.reset_index(drop=True, inplace=True)
print(completed_data_30_to_90_days)

       user_programs_user_id       date session_executions_updated_at  \
0                        280 2021-09-06    2021-09-06 16:57:35.623575   
1                        280 2021-09-07                             0   
2                        280 2021-09-08                             0   
3                        280 2021-09-09                             0   
4                        280 2021-09-10    2021-09-10 08:03:14.374092   
...                      ...        ...                           ...   
49405                  17017 2022-07-15                             0   
49406                  17017 2022-07-16                             0   
49407                  17017 2022-07-17                             0   
49408                  17017 2022-07-18                             0   
49409                  17017 2022-07-19                             0   

       1 leg bridge (left)_reps_1  1 leg bridge (left)_reps_10  \
0                             0.0                        

In [30]:
# Ensure that 'date' is in datetime format
df_filtered_90_to_180_days['date'] = pd.to_datetime(df_filtered_90_to_180_days['date'])

# Get the first training day for each user
first_training_day_90_to_180 = df_filtered_90_to_180_days.groupby('user_programs_user_id')['date'].min().reset_index()
first_training_day_90_to_180.columns = ['user_programs_user_id', 'first_training_date']

# Create an empty DataFrame to store the completed data
completed_data_90_to_180_days = pd.DataFrame()

# Iterate over each user and complete the missing days
for user_id, first_day in zip(first_training_day_90_to_180['user_programs_user_id'], first_training_day_90_to_180['first_training_date']):
    # Generate a date range from the first day up to 180 days later
    date_range = pd.date_range(start=first_day, periods=180)
    
    # Create a DataFrame with the date range and user_id
    user_dates = pd.DataFrame({'user_programs_user_id': user_id, 'date': date_range})
    
    # Merge with the original DataFrame to identify the days with and without training
    user_data = pd.merge(user_dates, df_filtered_90_to_180_days[df_filtered_90_to_180_days['user_programs_user_id'] == user_id], on=['user_programs_user_id', 'date'], how='left')
    
    # Fill NaN values (days without training) with zeros in the relevant columns
    user_data.fillna(0, inplace=True)
    
    # Add the completed data to the final DataFrame
    completed_data_90_to_180_days = pd.concat([completed_data_90_to_180_days, user_data])

# Reset the index of the final DataFrame
completed_data_90_to_180_days.reset_index(drop=True, inplace=True)
print(completed_data_90_to_180_days)

        user_programs_user_id       date session_executions_updated_at  \
0                         112 2021-10-08    2021-10-08 19:39:51.593682   
1                         112 2021-10-09                             0   
2                         112 2021-10-10                             0   
3                         112 2021-10-11                             0   
4                         112 2021-10-12                             0   
...                       ...        ...                           ...   
102955                  15966 2022-08-17                             0   
102956                  15966 2022-08-18                             0   
102957                  15966 2022-08-19                             0   
102958                  15966 2022-08-20                             0   
102959                  15966 2022-08-21                             0   

        1 leg bridge (left)_reps_1  1 leg bridge (left)_reps_10  \
0                              0.0          

In [31]:
# Ensure that 'date' is in datetime format
df_filtered_180_to_365_days['date'] = pd.to_datetime(df_filtered_180_to_365_days['date'])

# Get the first training day for each user
first_training_day_180_to_365 = df_filtered_180_to_365_days.groupby('user_programs_user_id')['date'].min().reset_index()
first_training_day_180_to_365.columns = ['user_programs_user_id', 'first_training_date']

# Create an empty DataFrame to store the completed data
completed_data_180_to_365_days = pd.DataFrame()

# Iterate over each user and complete the missing days
for user_id, first_day in zip(first_training_day_180_to_365['user_programs_user_id'], first_training_day_180_to_365['first_training_date']):
    # Generate a date range from the first day up to 365 days later
    date_range = pd.date_range(start=first_day, periods=365)
    
    # Create a DataFrame with the date range and user_id
    user_dates = pd.DataFrame({'user_programs_user_id': user_id, 'date': date_range})
    
    # Merge with the original DataFrame to identify the days with and without training
    user_data = pd.merge(user_dates, df_filtered_180_to_365_days[df_filtered_180_to_365_days['user_programs_user_id'] == user_id], on=['user_programs_user_id', 'date'], how='left')
    
    # Fill NaN values (days without training) with zeros in the relevant columns
    user_data.fillna(0, inplace=True)
    
    # Add the completed data to the final DataFrame
    completed_data_180_to_365_days = pd.concat([completed_data_180_to_365_days, user_data])

# Reset the index of the final DataFrame
completed_data_180_to_365_days.reset_index(drop=True, inplace=True)
print(completed_data_180_to_365_days)

        user_programs_user_id       date session_executions_updated_at  \
0                         108 2021-06-11    2021-06-11 18:00:35.640406   
1                         108 2021-06-12                             0   
2                         108 2021-06-13                             0   
3                         108 2021-06-14                             0   
4                         108 2021-06-15                             0   
...                       ...        ...                           ...   
149280                  11835 2022-11-22                             0   
149281                  11835 2022-11-23                             0   
149282                  11835 2022-11-24                             0   
149283                  11835 2022-11-25                             0   
149284                  11835 2022-11-26                             0   

        1 leg bridge (left)_reps_1  1 leg bridge (left)_reps_10  \
0                              0.0          

In [ ]:
# Save the DataFrames to a HDF5 file
completed_data_30days.to_hdf('../data/completed_data_30days_v1.h5', key='df')
completed_data_30_to_90_days.to_hdf('../data/completed_data_30_to_90_days_v1.h5', key='df')
completed_data_90_to_180_days.to_hdf('../data/completed_data_90_to_180_days_v1.h5', key='df')
completed_data_180_to_365_days.to_hdf('../data/completed_data_180_to_365_days_v1.h5', key='df')

In [42]:
# Save the DataFrames to a CSV file
completed_data_30days.to_csv('../data/completed_data_30_days.csv', index=False)
completed_data_30_to_90_days.to_csv('../data/completed_data_30_to_90_days.csv', index=False)
completed_data_90_to_180_days.to_csv('../data/completed_data_90_to_180_days.csv', index=False)
completed_data_180_to_365_days.to_csv('../data/completed_data_180_to_365_days.csv', index=False)

In [ ]:
# TODAVIA NO ESTA HECHO!!

# Read the datasets
completed_data_30days = pd.read_hdf('../data/completed_data_30days_v1.h5', key='df')
completed_data_30_to_90_days = pd.read_hdf('../data/completed_data_30_to_90_days_v1.h5', key='df')
completed_data_90_to_180_days = pd.read_hdf('../data/completed_data_90_to_180_days_v1.h5', key='df')
completed_data_180_to_365_days = pd.read_hdf('../data/completed_data_180_to_365_days_v1.h5', key='df')

print(f'30 days dataset shape:', completed_data_30days.shape)
print(f'30-90 days dataset shape:', completed_data_30_to_90_days.shape)
print(f'90-180 days dataset shape:', completed_data_90_to_180_days.shape)
print(f'180-365 days dataset shape:', completed_data_180_to_365_days.shape)

## **Unsupervised Problem into Time Series**

In [11]:
# Convert 'date' to datetime
completed_data_30days['date'] = pd.to_datetime(completed_data_30days['date'])

# Calculate the first and last training day for each user
user_training_period = completed_data_30days.groupby('user_programs_user_id')['date'].agg(['min', 'max']).reset_index()
user_training_period.columns = ['user_programs_user_id', 'first_training_date', 'last_training_date']

# Calculate the difference in days between the first and last training session
user_training_period['training_period_days'] = (user_training_period['last_training_date'] - user_training_period['first_training_date']).dt.days

# Merge the training period back to the original DataFrame
completed_data_30days = completed_data_30days.merge(user_training_period[['user_programs_user_id', 'training_period_days']], on='user_programs_user_id')

In [81]:
def create_time_series_features_multistep(df, user_col, date_col, feature_cols, target_col, window_size=7, n_steps=3):
    """
    Crea características de series temporales usando una ventana deslizante y predicción multistep.
    
    Args:
    df (DataFrame): DataFrame de entrada.
    user_col (str): Nombre de la columna del ID del usuario.
    date_col (str): Nombre de la columna de la fecha.
    feature_cols (list): Lista de nombres de columnas de características.
    target_col (str): Nombre de la columna objetivo.
    window_size (int): Tamaño de la ventana deslizante.
    n_steps (int): Número de pasos en el futuro a predecir.
    
    Returns:
    DataFrame: DataFrame con características de series temporales y etiquetas.
    """
    ts_data = []

    for user_id in df[user_col].unique():
        user_data = df[df[user_col] == user_id].reset_index(drop=True)
        for i in range(len(user_data) - window_size - n_steps + 1):
            features = user_data.loc[i:i+window_size-1, feature_cols].values.flatten()
            targets = user_data.loc[i+window_size:i+window_size+n_steps-1, target_col].values.flatten()
            ts_data.append([user_id, user_data.loc[i+window_size, date_col]] + list(features) + list(targets))
    
    columns = [user_col, date_col] + [f'{col}_t-{t}' for t in range(window_size-1, -1, -1) for col in feature_cols] + [f'{target_col}_t+{t+1}' for t in range(n_steps)]
    ts_df = pd.DataFrame(ts_data, columns=columns)
    
    return ts_df

# Obtener todas las columnas del DataFrame
all_columns = completed_data_30days.columns.tolist()

# Excluir las columnas 'user_programs_user_id', 'date' y 'training_days' para obtener las columnas de características
feature_cols = [col for col in all_columns if col not in ['user_programs_user_id', 'date', 'training_days']]

# Definir la columna objetivo (training_days)
target_col = 'training_period_days'

# Crear características de series temporales
ts_df_multistep = create_time_series_features_multistep(completed_data_30days, 'user_programs_user_id', 'date', feature_cols, target_col, window_size=7, n_steps=3)
print(ts_df_multistep.head())


In [79]:
def create_time_series_features_multistep_single_variable(df, user_col, date_col, feature_col, target_col, window_size=7, n_steps=3):
    """
    Crea características de series temporales usando una ventana deslizante y predicción multistep para una sola variable.
    
    Args:
    df (DataFrame): DataFrame de entrada.
    user_col (str): Nombre de la columna del ID del usuario.
    date_col (str): Nombre de la columna de la fecha.
    feature_col (str): Nombre de la columna de la característica.
    target_col (str): Nombre de la columna objetivo.
    window_size (int): Tamaño de la ventana deslizante.
    n_steps (int): Número de pasos en el futuro a predecir.
    
    Returns:
    DataFrame: DataFrame con características de series temporales y etiquetas.
    """
    ts_data = []

    for user_id in df[user_col].unique():
        user_data = df[df[user_col] == user_id].reset_index(drop=True)
        for i in range(len(user_data) - window_size - n_steps + 1):
            features = user_data.loc[i:i+window_size-1, feature_col].values.flatten()
            targets = user_data.loc[i+window_size:i+window_size+n_steps-1, target_col].values.flatten()
            ts_data.append([user_id, user_data.loc[i+window_size, date_col]] + list(features) + list(targets))
    
    columns = [user_col, date_col] + [f'{feature_col}_t-{t}' for t in range(window_size-1, -1, -1)] + [f'{target_col}_t+{t+1}' for t in range(n_steps)]
    ts_df = pd.DataFrame(ts_data, columns=columns)
    
    return ts_df

# Definir la columna de características y la columna objetivo
feature_col = 'session_executions_updated_at'
target_col = 'training_period_days'

# Crear características de series temporales
ts_df_multistep_single_variable = create_time_series_features_multistep_single_variable(completed_data_30days, 'user_programs_user_id', 'date', feature_col, target_col, window_size=7, n_steps=3)

# Mostrar las primeras filas del DataFrame resultante
print(ts_df_multistep_single_variable.head())


   user_programs_user_id       date session_executions_updated_at_t-6  \
0                    172 2021-11-20        2021-11-13 10:11:42.357218   
1                    172 2021-11-21        2021-11-14 12:05:10.670652   
2                    172 2021-11-22        2021-11-15 12:53:48.363087   
3                    172 2021-11-23                                 0   
4                    172 2021-11-24                                 0   

  session_executions_updated_at_t-5 session_executions_updated_at_t-4  \
0        2021-11-14 12:05:10.670652        2021-11-15 12:53:48.363087   
1        2021-11-15 12:53:48.363087                                 0   
2                                 0                                 0   
3                                 0                                 0   
4                                 0                                 0   

  session_executions_updated_at_t-3 session_executions_updated_at_t-2  \
0                                 0              

In [13]:
def create_time_series_features_multistep_two_variables(df, user_col, date_col, feature_cols, target_col, window_size=7, n_steps=3):
    """
    Crea características de series temporales usando una ventana deslizante y predicción multistep para dos variables.
    
    Args:
    df (DataFrame): DataFrame de entrada.
    user_col (str): Nombre de la columna del ID del usuario.
    date_col (str): Nombre de la columna de la fecha.
    feature_cols (list): Lista de nombres de columnas de características.
    target_col (str): Nombre de la columna objetivo.
    window_size (int): Tamaño de la ventana deslizante.
    n_steps (int): Número de pasos en el futuro a predecir.
    
    Returns:
    DataFrame: DataFrame con características de series temporales y etiquetas.
    """
    ts_data = []

    for user_id in df[user_col].unique():
        user_data = df[df[user_col] == user_id].reset_index(drop=True)
        for i in range(len(user_data) - window_size - n_steps + 1):
            features = user_data.loc[i:i+window_size-1, feature_cols].values.flatten()
            targets = user_data.loc[i+window_size:i+window_size+n_steps-1, target_col].values.flatten()
            ts_data.append([user_id, user_data.loc[i+window_size, date_col]] + list(features) + list(targets))
    
    columns = [user_col, date_col] + [f'{col}_t-{t}' for t in range(window_size-1, -1, -1) for col in feature_cols] + [f'{target_col}_t+{t+1}' for t in range(n_steps)]
    ts_df = pd.DataFrame(ts_data, columns=columns)
    
    return ts_df

# Definir las columnas de características y la columna objetivo
feature_cols = ['session_executions_updated_at', 'sum_of_reps']
target_col = 'training_period_days'

# Crear características de series temporales
ts_df_multistep_two_variables = create_time_series_features_multistep_two_variables(completed_data_30days, 'user_programs_user_id', 'date', feature_cols, target_col, window_size=7, n_steps=3)

# Mostrar las primeras filas del DataFrame resultante
print(ts_df_multistep_two_variables.head())

   user_programs_user_id       date session_executions_updated_at_t-6  \
0                    172 2021-11-20        2021-11-13 10:11:42.357218   
1                    172 2021-11-21        2021-11-14 12:05:10.670652   
2                    172 2021-11-22        2021-11-15 12:53:48.363087   
3                    172 2021-11-23                                 0   
4                    172 2021-11-24                                 0   

   sum_of_reps_t-6 session_executions_updated_at_t-5  sum_of_reps_t-5  \
0            140.0        2021-11-14 12:05:10.670652            183.0   
1            183.0        2021-11-15 12:53:48.363087            220.0   
2            220.0                                 0              0.0   
3              0.0                                 0              0.0   
4              0.0                                 0              0.0   

  session_executions_updated_at_t-4  sum_of_reps_t-4  \
0        2021-11-15 12:53:48.363087            220.0   
1         

In [14]:
ts_df_multistep_two_variables.head()

,user_programs_user_id,date,session_executions_updated_at_t-6,sum_of_reps_t-6,session_executions_updated_at_t-5,sum_of_reps_t-5,session_executions_updated_at_t-4,sum_of_reps_t-4,session_executions_updated_at_t-3,sum_of_reps_t-3,session_executions_updated_at_t-2,sum_of_reps_t-2,session_executions_updated_at_t-1,sum_of_reps_t-1,session_executions_updated_at_t-0,sum_of_reps_t-0,training_period_days_t+1,training_period_days_t+2,training_period_days_t+3
0,172,2021-11-20,2021-11-13 10:11:42.357218,140.0,2021-11-14 12:05:10.670652,183.0,2021-11-15 12:53:48.363087,220.0,0,0.0,0,0.0,0,0.0,0,0.0,29,29,29
1,172,2021-11-21,2021-11-14 12:05:10.670652,183.0,2021-11-15 12:53:48.363087,220.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,29,29,29
2,172,2021-11-22,2021-11-15 12:53:48.363087,220.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,29,29,29
3,172,2021-11-23,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,2021-11-22 20:01:00.771300,215.0,29,29,29
4,172,2021-11-24,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,2021-11-22 20:01:00.771300,215.0,0,0.0,29,29,29


In [80]:
ts_df_multistep_single_variable.head()

,user_programs_user_id,date,session_executions_updated_at_t-6,session_executions_updated_at_t-5,session_executions_updated_at_t-4,session_executions_updated_at_t-3,session_executions_updated_at_t-2,session_executions_updated_at_t-1,session_executions_updated_at_t-0,training_period_days_t+1,training_period_days_t+2,training_period_days_t+3
0,172,2021-11-20,2021-11-13 10:11:42.357218,2021-11-14 12:05:10.670652,2021-11-15 12:53:48.363087,0,0,0,0,29,29,29
1,172,2021-11-21,2021-11-14 12:05:10.670652,2021-11-15 12:53:48.363087,0,0,0,0,0,29,29,29
2,172,2021-11-22,2021-11-15 12:53:48.363087,0,0,0,0,0,0,29,29,29
3,172,2021-11-23,0,0,0,0,0,0,2021-11-22 20:01:00.771300,29,29,29
4,172,2021-11-24,0,0,0,0,0,2021-11-22 20:01:00.771300,0,29,29,29


In [63]:
completed_data_30days.head(3)

,user_programs_user_id,date,session_executions_updated_at,1 leg bridge (left)_reps_1,1 leg bridge (left)_reps_10,1 leg bridge (left)_reps_11,1 leg bridge (left)_reps_12,1 leg bridge (left)_reps_13,1 leg bridge (left)_reps_14,1 leg bridge (left)_reps_15,1 leg bridge (left)_reps_16,1 leg bridge (left)_reps_17,1 leg bridge (left)_reps_2,1 leg bridge (left)_reps_3,1 leg bridge (left)_reps_4,1 leg bridge (left)_reps_5,1 leg bridge (left)_reps_6,1 leg bridge (left)_reps_7,1 leg bridge (left)_reps_8,1 leg bridge (left)_reps_9,1 leg bridge (left)_time_1,1 leg bridge (left)_time_10,1 leg bridge (left)_time_11,1 leg bridge (left)_time_12,1 leg bridge (left)_time_13,1 leg bridge (left)_time_14,1 leg bridge (left)_time_15,1 leg bridge (left)_time_16,1 leg bridge (left)_time_17,1 leg bridge (left)_time_2,1 leg bridge (left)_time_3,1 leg bridge (left)_time_4,1 leg bridge (left)_time_5,1 leg bridge (left)_time_6,1 leg bridge (left)_time_7,1 leg bridge (left)_time_8,1 leg bridge (left)_time_9,1 leg bridge (right)_reps_1,1 leg bridge (right)_reps_10,1 leg bridge (right)_reps_11,1 leg bridge (right)_reps_12,1 leg bridge (right)_reps_13,1 leg bridge (right)_reps_14,1 leg bridge (right)_reps_15,1 leg bridge (right)_reps_16,1 leg bridge (right)_reps_17,1 leg bridge (right)_reps_2,1 leg bridge (right)_reps_3,1 leg bridge (right)_reps_4,1 leg bridge (right)_reps_5,1 leg bridge (right)_reps_6,1 leg bridge (right)_reps_7,1 leg bridge (right)_reps_8,1 leg bridge (right)_reps_9,1 leg bridge (right)_time_1,1 leg bridge (right)_time_10,1 leg bridge (right)_time_11,1 leg bridge (right)_time_12,1 leg bridge (right)_time_13,1 leg bridge (right)_time_14,1 leg bridge (right)_time_15,1 leg bridge (right)_time_16,1 leg bridge (right)_time_17,1 leg bridge (right)_time_2,1 leg bridge (right)_time_3,1 leg bridge (right)_time_4,1 leg bridge (right)_time_5,1 leg bridge (right)_time_6,1 leg bridge (right)_time_7,1 leg bridge (right)_time_8,1 leg bridge (right)_time_9,1 leg dead lift (left)_reps_1,1 leg dead lift (left)_reps_10,1 leg dead lift (left)_reps_11,1 leg dead lift (left)_reps_12,1 leg dead lift (left)_reps_13,1 leg dead lift (left)_reps_14,1 leg dead lift (left)_reps_15,1 leg dead lift (left)_reps_2,1 leg dead lift (left)_reps_3,1 leg dead lift (left)_reps_4,1 leg dead lift (left)_reps_5,1 leg dead lift (left)_reps_6,1 leg dead lift (left)_reps_7,1 leg dead lift (left)_reps_8,1 leg dead lift (left)_reps_9,1 leg dead lift (left)_time_1,1 leg dead lift (left)_time_10,1 leg dead lift (left)_time_11,1 leg dead lift (left)_time_12,1 leg dead lift (left)_time_13,1 leg dead lift (left)_time_14,1 leg dead lift (left)_time_15,1 leg dead lift (left)_time_2,1 leg dead lift (left)_time_3,1 leg dead lift (left)_time_4,1 leg dead lift (left)_time_5,1 leg dead lift (left)_time_6,1 leg dead lift (left)_time_7,1 leg dead lift (left)_time_8,1 leg dead lift (left)_time_9,1 leg dead lift (right)_reps_1,1 leg dead lift (right)_reps_10,1 leg dead lift (right)_reps_11,1 leg dead lift (right)_reps_12,1 leg dead lift (right)_reps_13,1 leg dead lift (right)_reps_14,1 leg dead lift (right)_reps_15,1 leg dead lift (right)_reps_2,1 leg dead lift (right)_reps_3,1 leg dead lift (right)_reps_4,1 leg dead lift (right)_reps_5,1 leg dead lift (right)_reps_6,1 leg dead lift (right)_reps_7,1 leg dead lift (right)_reps_8,1 leg dead lift (right)_reps_9,1 leg dead lift (right)_time_1,1 leg dead lift (right)_time_10,1 leg dead lift (right)_time_11,1 leg dead lift (right)_time_12,1 leg dead lift (right)_time_13,1 leg dead lift (right)_time_14,1 leg dead lift (right)_time_15,1 leg dead lift (right)_time_2,1 leg dead lift (right)_time_3,1 leg dead lift (right)_time_4,1 leg dead lift (right)_time_5,1 leg dead lift (right)_time_6,1 leg dead lift (right)_time_7,1 leg dead lift (right)_time_8,1 leg dead lift (right)_time_9,1 leg dead lift 1with KB (right)_reps_1,1 leg dead lift 1with KB (right)_reps_10,1 leg dead lift 1with KB (right)_reps_11,1 leg dead lift 1with KB (right)_reps_12,1 leg dead lift 1with K